In [3]:
# =========================
# Cell 0 - Imports & config
# =========================
from pathlib import Path
import os
import pandas as pd
import matplotlib.pyplot as plt

SIDE_LENGTH = 4  # <- set this

base = f"tfim_{SIDE_LENGTH}x{SIDE_LENGTH}_magnetizations"
paths = {
    "full":       Path(f"{base}_full.csv"),
    "three_point":Path(f"{base}_three_point.csv"),
    "one_point":  Path(f"{base}_one_point.csv"),
    "ref":        Path(f"{base}_ref.csv"),
}

def _load_csv(p: Path) -> pd.DataFrame:
    if p.exists():
        return pd.read_csv(p)
    print(f"[warn] missing: {p}")
    return None

df_full  = _load_csv(paths["full"])
df_three = _load_csv(paths["three_point"])
df_one   = _load_csv(paths["one_point"])
ref_df   = _load_csv(paths["ref"])

In [5]:
# =========================
# Cell 1 - Plot helper
# =========================
def plot_magnetizations_panel(ax, df, ref_df, title: str):
    if df is None:
        ax.text(0.5, 0.5, "missing data file", ha="center", va="center", transform=ax.transAxes)
        ax.set_title(title)
        ax.grid(True, alpha=0.3)
        return

    supp = df[df["type"] == "support"]
    nov  = df[df["type"] == "novel"]

    # Reference curves (if available)
    if ref_df is not None:
        ax.plot(ref_df["h"], ref_df["mag_z"], ":", color="gray", linewidth=2.0,
                label="_nolegend_", zorder=1)
        ax.plot(ref_df["h"], ref_df["mag_x"], ":", color="gray", linewidth=2.0,
                label=r'$\langle \sigma^{z/x} \rangle_{\mathrm{reference}}$', zorder=1)

    # Support
    ax.plot(supp["h"], supp["mz"], "o", color="C0", markersize=8,
            label=r'$\langle \sigma^z \rangle_{\mathrm{support}}$', zorder=2)
    ax.plot(supp["h"], supp["mx"], "D", color="C1", markersize=8,
            label=r'$\langle \sigma^x \rangle_{\mathrm{support}}$', zorder=2)

    # Novel
    ax.plot(nov["h"], nov["mz"], "o", color="C0", markerfacecolor="none",
            markersize=8, markeredgewidth=1.5,
            label=r'$\langle \sigma^z \rangle_{\mathrm{novel}}$', zorder=2)
    ax.plot(nov["h"], nov["mx"], "D", color="C1", markerfacecolor="none",
            markersize=8, markeredgewidth=1.5,
            label=r'$\langle \sigma^x \rangle_{\mathrm{novel}}$', zorder=2)

    ax.set_title(title)
    ax.set_xlabel(r"$h$")
    ax.grid(True, alpha=0.3)




fig, axes = plt.subplots(
    1, 3,
    figsize=(14, 3),
    sharey=True,
    gridspec_kw={"wspace": 0.06}
)

plot_magnetizations_panel(axes[0], df_full,  ref_df, f"TFIM ({SIDE_LENGTH}x{SIDE_LENGTH}) - full")
plot_magnetizations_panel(axes[1], df_three, ref_df, f"TFIM ({SIDE_LENGTH}x{SIDE_LENGTH}) - three_point")
plot_magnetizations_panel(axes[2], df_one,   ref_df, f"TFIM ({SIDE_LENGTH}x{SIDE_LENGTH}) - one_point")

axes[0].set_ylabel("Magnetization")

# Keep it tight: only one legend (left panel), but same plotting logic everywhere
handles, labels = axes[0].get_legend_handles_labels()
axes[0].legend(handles, labels, frameon=True, fontsize=10, loc="best")

plt.tight_layout()

plt.show()
